In [ ]:
# Import necessary packages
# Requirements:
# (1) pytorch 
# (2) numpy 
# (3) matplotlib
# (4) pandas

import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import csv

print(os.getcwd())

In [ ]:
# Define speeds (m/s) corresponding to 0-40 Hz settings in wind tunnel
### (Just for record-keeping)
hzArray = np.array((0, 5, 10, 15, 20, 25, 30, 35, 40))
speedArray = np.array((0.00, 1.26363735, 1.58562983, 2.07066356, 2.571993, 3.18291372, 3.75322345, 4.33626595, 4.91413509))
###



In [ ]:
class CrossWireDataset(Dataset):
    '''
    Dataset class for pytorch-based learning tailored to crosswire model training. This method 
    essentially is feature learning of a specific, reduced set of features from the sensor readings, 
    namely: 

    [Input Features]
       --- The maximal (absolute) voltage reading (voltage)
       --- The index of the maximal (absolute) voltage reading (integer, {1-6})
       --- The (regularized) ratio of the adjacent sensors (voltage/voltage)
    [Predictions]
       --- The gust speed (m/s)
       --- The gust incident angle (radians)

    '''
    def __init__(self, magFile, angFile, readingsFile, transform=None, target_transform=None):
        # Construct the labels
        tmpMag = pd.read_csv(magFile)
        tmpAng = pd.read_csv(angFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.angs = torch.Tensor(tmpAng.to_numpy())
        
        # Construct the features and place them into readings array(X). 
        tmpReadings = pd.read_csv(readingsFile)
        tmpReadings = tmpReadings.to_numpy()
        print(tmpReadings.shape)
        LL = tmpReadings.shape[0]
        tmpReadings2 = np.zeros((LL, 3))
        for k in range(LL):
            tmpReadings2[k, 0] = np.max(np.abs(tmpReadings[k, :]))
            tmpReadings2[k, 1] = np.argmax(np.abs(tmpReadings[k, :]))
            tt = int(tmpReadings2[k,1])
            tmpReadings2[k, 2] = np.abs(tmpReadings[k, (tt-1)%6])/(np.abs(tmpReadings[k, (tt+1)%6]) + 0.05)
        
        self.readings = torch.Tensor(tmpReadings2)
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        mag = self.mags[idx]
        ang = self.angs[idx]
        label = torch.cat((mag, ang), 0)
        
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

In [ ]:
class WindMagDataset(Dataset):
    '''
    Dataset class for pytorch-based learning for gust magnitude data training. This method 
    learns directly from the sensor readings (voltages) to predict gust speed (m/s). 
    [Inputs]
       --- The sensor readings (voltages)
    [Predictions]
       --- The gust speed (m/s)
    '''
    def __init__(self, magFile, readingsFile, transform=None, target_transform=None):
        tmpMag = pd.read_csv(magFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.mags = torch.Tensor(tmpMag.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.mags)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        label = self.mags[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

class WindAngDataset(Dataset):
    '''
    Dataset class for pytorch-based learning for gust angle data training. This method 
    learns directly from the sensor readings (voltages) to predict gust incidence angle (radians). 
    [Inputs]
       --- The sensor readings (voltages)
    [Predictions]
       --- The gust angle (rad)
    '''
    def __init__(self, angFile, readingsFile, transform=None, target_transform=None):
        tmpAng = pd.read_csv(angFile)
        tmpReadings = pd.read_csv(readingsFile)
        self.angs = torch.Tensor(tmpAng.to_numpy())
        self.readings = torch.Tensor(tmpReadings.to_numpy())
        
        # Incorporate the transforms as needed
        self.transform=transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.angs)
    
    def __getitem__(self, idx):
        reading = self.readings[idx, :]
        label = self.angs[idx]
        if self.transform:
            reading = self.transform(reading)
        if self.target_transform:
            label = self.target_transform(label)

        return reading, label

In [ ]:
class NeuralNetwork(nn.Module):
    '''
    A general/generic Neural Network model class for use with Pytorch. 
    
    TODO: include layer widths, types, and nonlinearities as inputs and dynamically allocate
          --- this will allow for custom classes rather than the clunky "if" statement used here. 
    '''
    def __init__(self, crosswire=False, fullAngles=False, geom=6):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        
        if crosswire:
            # Architecture to use for crosswire prediction
            # Input size is 3  --- three crosswire features
            # Output size is 2 --- speed (m/s) and angle (rad)
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(3, 25),
                nn.ReLU(),
                nn.Linear(25, 15),
                nn.ReLU(),
                nn.Linear(15, 2),
            )
        else:
            if fullAngles:
                # Architecture to use for angle prediction if the data is dense (2-degree increments)
                # Input size is 6  --- six sensor readings (voltages)
                # Output size is 1 --- angle (rad)
                self.linear_relu_stack = nn.Sequential(
                    nn.Linear(geom, 30),
                    nn.ReLU(),
                    nn.Linear(30, 18),
                    nn.ReLU(),
                    nn.Linear(18, 1),
                )
            else:
                # Architecture to use for speed prediction (generally) and for angle prediction 
                # if the data is NOT dense (e.g., is in 10-degree increments)
                # Input size is 6  --- six sensor readings (voltages)
                # Output size is 1 --- either speed (m/s) or angle (rad)
                self.linear_relu_stack = nn.Sequential(
                    nn.Linear(geom, 50),
                    nn.ReLU(),
                    nn.Linear(50, 25),
                    nn.ReLU(),
                    nn.Linear(25, 1),
                )

    def forward(self, x):
        # Method to propagate input (reading) through the network to get a prediction. 
        # Terminology is clunky because this is adapted from a classification example, hence 
        # the use of 'logits' even though we are doing regression.
        
        # TODO -- tidy up variable names, usage, etc (see above)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
def train_loop(dataloader, model, optimizer, epochNum, seedNum, loss_fn=nn.L1Loss(), verbose=True, batch_size=180, writePath=None):
    """
    Loop for training a 'model' (class NeuralNetwork) on data stored in 'dataloader,' using loss function
    'loss_fn' and optimizer method 'optimizer'
    
    [Inputs]
    dataloader    -- type DataLoader    -- Pytorch DataLoader object to facilitate training/testing data storage
                                           to interface with pytorch optimization and training modules
                                           
    model         -- type NeuralNetwork -- Pytorch NeuralNetwork object to facilitate training/testing of speed and 
                                           angle prediction for FlowDrone
                                           
    epochNum      -- type int           -- Current epoch number to track training loss
    
    seedNum       -- type int           -- Seed of the current run (to average over to demonstrate convergence)
    
    loss_fn       -- type torch.nn loss -- Pytorch loss function (in nn library) for training the speed/angle predictor
                                           for FlowDrone. Defaults to nn.MSELoss() because we are regressing real-valued
                                           variables. 
                                           
    optimizer     -- type torch.optim   -- Pytorch optimizer for ANN weight updates. Normally will use ADAM unless there
                                           is a compelling reason to deviate. 
    
    verbose       -- type Boolean       -- Toggles printing of training loss during training. Default is TRUE. 
    
    writePath     -- type String        -- If present, a path to write to a csv file in order 
    [Outputs]
    
    None

    """
    size = len(dataloader.dataset)
    f = open(writePath+'trainCostInit'+str(seedNum)+'.csv', 'a')
    writer = csv.writer(f)
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        for k in range(pred.shape[0]):
            if (pred[k, -1] >= 330.0*math.pi/180.0 and y[k, -1] <= 30.0*math.pi/180.0):
                pred[k, -1] += -2.0*math.pi
            elif (pred[k, -1] <= 30.0*math.pi/180.0 and y[k, -1] >= 330.0*math.pi/180.0):
                pred[k, -1] += 2.0*math.pi

        loss = loss_fn(pred, y)
        loss_average = (torch.sum(loss)/pred.shape[0]).cpu().detach().numpy()
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 200 == 1:
            if verbose:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            if writePath is None:
                pass
            else: 
                writer.writerow(np.array([(batch + (np.ceil(size/batch_size)*epochNum)), 180.0*loss_average/math.pi]))
                

    # close the file
    f.close()
        


def test_loop(dataloader, model, epochNum, seedNum, loss_fn=nn.L1Loss(), lastLoop=False, writePath=None):
    """
    Loop for test a 'model' (class NeuralNetwork) on data stored in 'dataloader,' using loss function
    'loss_fn.'
    
    [Inputs]
    dataloader    -- type DataLoader    -- Pytorch DataLoader object to facilitate training/testing data storage
                                           to interface with pytorch optimization and training modules
                                           
    model         -- type NeuralNetwork -- Pytorch NeuralNetwork object to facilitate training/testing of speed and 
                                           angle prediction for FlowDrone

    epochNum      -- type Int           -- Current epoch
    
    loss_fn       -- type torch.nn loss -- Pytorch loss function (in nn library) for training the speed/angle predictor
                                           for FlowDrone. Defaults to nn.MSELoss() because we are regressing real-valued
                                           variables. 
                                           
    lastLoop      -- type Boolean       -- Whether we are on the last epoch (for histogram information)
    
    writePath     -- type String        -- File to write to
    [Outputs]
    
    None

    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print(num_batches)
    hists = False
    
    if lastLoop:
        if (num_batches == size):
            errs = np.zeros(size)
            idxVal = 0
            hists = True
        
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            for k in range(pred.shape[0]):
                if (pred[k, -1] >= 330.0*math.pi/180.0 and y[k, -1] <= 30.0*math.pi/180.0):
                    pred[k, -1] += -2.0*math.pi
                elif (pred[k, -1] <= 30.0*math.pi/180.0 and y[k, -1] >= 330.0*math.pi/180.0):
                    pred[k, -1] += 2.0*math.pi
                
            if hists:
                errs[idxVal] = loss_fn(pred, y).item()
                test_loss += errs[idxVal]
                idxVal += 1
            else:
                test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")
    print(f"Avg error (deg): {(test_loss*180.0/math.pi):>8f} \n")
    
    if writePath is None:
        pass
    else:
        # open the file
        f = open(writePath+'testCostInit'+str(seedNum)+'.csv', 'a')
        writer = csv.writer(f)
        
        # write out the relevant cost
        writer.writerow(np.array([epochNum, test_loss*180.0/math.pi]))
                
        # close the file
        f.close()
    
    if hists:
        plt.figure(1)
        plt.hist(errs)
        plt.show()
        
        return test_loss, errs
    else: 
        return test_loss

In [ ]:
def makeDataset(dataSetType=2, geometryVal=3, compFlag=True, N=1, epochs0=15):
    # Make dataset
    # Change this as desired in {1, 2, 3, 4, 5}
    #
    # INDEX: 
    #   --- (1) Sparse wind magnitudes                         [OLD]
    #   --- (2) Sparse wind angles (10-degree increments)      [OLD]
    #   --- (3) Dense Crosswire Model 
    #   --- (4) Dense Wind Magnitudes 
    #   --- (5) Dense Incidence Angles (2-degree increments)
    
    ### dataSetType = 2

    # 3=Triangle, 4=Square, 5=Pentagon, 6=Hexagon
    ### geometryVal = 3
    ### compFlag=True
    ### N = 1                 # Number of sequentially averaged data points

    if geometryVal == 3:
        geomPath='tri/'
    elif geometryVal == 4:
        geomPath='squ/'
    elif geometryVal == 5:
        geomPath='pent/'
    elif geometryVal == 6:
        geomPath='hex/'
    else:
        raise ValueError('Geometry must be in {3, 4, 5, 6}')

    trainLabelPath = 'compTrain/'
    testLabelPath = 'compVal/'
    trainPath='compTrain_N'+str(N)+'/'
    testPath='compVal_N'+str(N)+'/'       # Set to validation data for network/hyperparameter optimization, else test data

    # Don't change these; the 'if' statements take care of them
    # Set network parameters in NeuralNetwork class
    fullAnglesVal = False
    crosswireVal = False

    if dataSetType==1:
        if compFlag:
            trainY = trainLabelPath+'mags.csv'
            trainX = trainPath+geomPath+'readings.csv'
            testY = testLabelPath+'mags.csv'
            testX = testPath+geomPath+'readings.csv'
        else:
            trainY = 'MagTrain/mags.csv'
            trainX = 'MagTrain/readings.csv'
            testY = 'MagTest/mags.csv'
            testX = 'MagTest/readings.csv'

        training_data = WindMagDataset(trainY, trainX, transform=None)
        testing_data = WindMagDataset(testY, testX, transform=None)
        epochs = epochs0

    elif dataSetType==2:
        if compFlag:
            trainY = trainLabelPath+'angsrad.csv'
            trainX = trainPath+geomPath+'readings.csv'
            testY = testLabelPath+'angsrad.csv'
            testX = testPath+geomPath+'readings.csv'
            fullAnglesVal = True

        else:
            trainY = 'MagTrain/angsrad.csv'
            trainX = 'MagTrain/readings.csv'
            testY = 'MagTest/angsrad.csv'
            testX = 'MagTest/readings.csv'

        training_data = WindAngDataset(trainY, trainX, transform=None)
        testing_data = WindAngDataset(testY, testX, transform=None)
        epochs = epochs0

    elif dataSetType==3:
        trainY1 = 'CrossTrain/crossmags.csv'
        trainY2 = 'CrossTrain/crossangsrad.csv'
        trainX = 'CrossTrain/crossreadings.csv'
        testY1 = 'CrossTest/crossmags.csv'
        testY2 = 'CrossTest/crossangsrad.csv'
        testX = 'CrossTest/crossreadings.csv'

        training_data = CrossWireDataset(trainY1, trainY2, trainX, transform=None)
        testing_data = CrossWireDataset(testY1, testY2, testX, transform=None)
        epochs = 2*epochs0

        crosswireVal = True

    elif dataSetType==4:

        trainY = 'CrossTrain/crossmags.csv'
        trainX = 'CrossTrain/crossreadings.csv'
        testY = 'CrossTest/crossmags.csv'
        testX = 'CrossTest/crossreadings.csv'

        training_data = WindMagDataset(trainY, trainX, transform=None)
        testing_data = WindMagDataset(testY, testX, transform=None)
        epochs = epochs0

    elif dataSetType==5:
        trainY = 'CrossTrain/crossangsrad.csv'
        trainX = 'CrossTrain/crossreadings.csv'
        testY = 'CrossTest/crossangsrad.csv'
        testX = 'CrossTest/crossreadings.csv'

        training_data = WindAngDataset(trainY, trainX, transform=None)
        testing_data = WindAngDataset(testY, testX, transform=None)
        epochs = epochs0

        fullAnglesVal = True

    else:
        raise ValueError('Not a valid dataSetType index (must be in {1, 2, 3, 4, or 5})')

    '''
    # Make training and testing data
    train_dataloader = DataLoader(training_data, batch_size=180, shuffle=True)
    test_dataloader = DataLoader(testing_data, batch_size=72, shuffle=True)
    model = NeuralNetwork(crosswire=crosswireVal, fullAngles=fullAnglesVal, geom=geometryVal)
    opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
    '''
    
    return training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, trainLabelPath, testPath, testLabelPath, geomPath

In [ ]:
# Define needed learning quantities

# Learning rate (initial)
# ### Generally ~ 1e-3 for Adam
learning_rate = 1e-3

# Batch size (default 64)
batch_size = 180

# Number of training epochs for 
# ### the simpler regression problems
epochs0 = 15

# Loss Function (MSE/MAE usually because we are 
# running relatively standard regression)

### ### Mean Squared error loss
# loss_fn = nn.MSELoss()

# Mean Absolute Error Loss
loss_fn = nn.L1Loss()

# Verbose flag toggles training
verboseFlag=False

# training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, testPath = makeDataset(dataSetType=2, geometryVal=3, compFlag=True, N=1)

In [ ]:
NN = np.array([1, 2, 5])
GEOMVAL = np.arange(3, 7)

for ii in range(len(NN)):
    for jj in range(len(GEOMVAL)):
        N = NN[ii]
        geometryVal = GEOMVAL[jj]
        for kk in range(5):
            np.random.seed(kk*12345 + 31415*N*(geometryVal**3))
            # Make all necessary data
            training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, trainLabelPath, testPath, testLabelPath, geomPath = makeDataset(dataSetType=2, geometryVal=geometryVal, compFlag=True, N=N, epochs0=epochs0)
            
            # Make training and testing dataloaders
            # Initialize model and optimizer params
            train_dataloader = DataLoader(training_data, batch_size=180, shuffle=True)
            test_dataloader = DataLoader(testing_data, batch_size=72, shuffle=True)
            model = NeuralNetwork(crosswire=crosswireVal, fullAngles=fullAnglesVal, geom=geometryVal)
            opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
            avg_error = test_loop(test_dataloader, model, 0, kk, loss_fn, lastLoop=False, writePath=(testPath+geomPath))
            
            '''
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loop(train_dataloader, model, opt, t, kk, loss_fn, verbose=verboseFlag, writePath=(testPath+geomPath))
                print()
                # if (float(t+1)/float(epochs) >= k or (t==(epochs-1))):
                if t < epochs - 1:
                    avg_error = test_loop(test_dataloader, model, t, kk, loss_fn, lastLoop=(t==(epochs-1)), writePath=(testPath+geomPath))
                else: 
                    # avg_error, Z = test_loop(test_dataloader, model, loss_fn, lastLoop=(t==(epochs-1)))
                    avg_error = test_loop(test_dataloader, model, t, kk, loss_fn, lastLoop=(t==(epochs-1)), writePath=(testPath+geomPath))
            '''
            # Print that we have finished training
            print(f"Finished seed: {kk+1:>2f} of 5, on geometry {jj+1:>2f} of 4, on filtering setting {ii+1:>2f} of 3 \n")

            


In [ ]:
# Triangle -- Best: ~20.6 degrees mean error; ~30 epochs; network layers 3-30-15-1; 
#          -- tested +10 more epochs and stalled in 20.9-21.5 test error range
#
# Square   -- ~4.5 degrees mean error; ~330 epochs; network layers [4]-50-25-1
#          -- tested +00 more epochs, stalled in 6-6.5 range
#

In [ ]:
Zs = np.sort(Z)
np.savetxt(testPath+geomPath+'ValErrorSort.csv', Zs, delimiter=',')

print('Mean Absolute Error (deg): ', str(np.mean(Zs)*180.0/math.pi))

# Choose what fraction of Zs to study
fracPred = 0.98


# Take the >fracPred set of best predictions
nKeep = int(np.ceil(fracPred*len(Zs)))
Zsmall = Zs[:nKeep]

# Print the fracPred quantile worst prediction
print(np.max(Zsmall))
print('This is equivalent to '+ str(np.max(Zsmall)*180.0/math.pi) + ' degrees')
print('Mean Absolute Error, best 98% (deg): ', str(np.mean(Zsmall)*180.0/math.pi))

In [ ]:
plt.figure(2)
plt.hist(Zsmall, cumulative=False, density=True, bins=20)
plt.show()

In [ ]:
zs = int(len(Zsmall)*0.84)
print(zs)

In [ ]:
print(Zsmall.shape)
print(Zsmall[zs]*180/math.pi)

In [ ]:
NN = np.array([1, 2, 5])
print(NN[0])
print(NN.shape)
GEOMVAL = np.arange(3, 7)
print(GEOMVAL)
print(GEOMVAL.shape)
# training_data, testing_data, epochs, fullAnglesVal, crosswireVal, trainPath, testPath = makeDataset(dataSetType=2, geometryVal=3, compFlag=True, N=1)

In [ ]:
# Make dataset
# Change this as desired in {1, 2, 3, 4, 5}
#
# INDEX: 
#   --- (1) Sparse wind magnitudes                         [OLD]
#   --- (2) Sparse wind angles (10-degree increments)      [OLD]
#   --- (3) Dense Crosswire Model 
#   --- (4) Dense Wind Magnitudes 
#   --- (5) Dense Incidence Angles (2-degree increments)
dataSetType = 2

# 3=Triangle, 4=Square, 5=Pentagon, 6=Hexagon
geometryVal = 3
compFlag=True
N = 1                 # Number of sequentially averaged data points

if geometryVal == 3:
    geomPath='tri/'
elif geometryVal == 4:
    geomPath='squ/'
elif geometryVal == 5:
    geomPath='pent/'
elif geometryVal == 6:
    geomPath='hex/'
else:
    raise ValueError('Geometry must be in {3, 4, 5, 6}')

trainPath='compTrain_N'+str(N)+'/'
testPath='compVal_N'+str(N)+'/'       # Set to validation data for network/hyperparameter optimization, else test data

# Don't change these; the 'if' statements take care of them
# Set network parameters in NeuralNetwork class
fullAnglesVal = False
crosswireVal = False

if dataSetType==1:
    if compFlag:
        trainY = trainPath+geomPath+'mags.csv'
        trainX = trainPath+geomPath+'readings.csv'
        testY = testPath+geomPath+'mags.csv'
        testX = testPath+geomPath+'readings.csv'
    else:
        trainY = 'MagTrain/mags.csv'
        trainX = 'MagTrain/readings.csv'
        testY = 'MagTest/mags.csv'
        testX = 'MagTest/readings.csv'
    
    training_data = WindMagDataset(trainY, trainX, transform=None)
    testing_data = WindMagDataset(testY, testX, transform=None)
    epochs = epochs0
    
elif dataSetType==2:
    if compFlag:
        trainY = trainPath+geomPath+'angsrad.csv'
        trainX = trainPath+geomPath+'readings.csv'
        testY = testPath+geomPath+'angsrad.csv'
        testX = testPath+geomPath+'readings.csv'
        fullAnglesVal = True
        
    else:
        trainY = 'MagTrain/angsrad.csv'
        trainX = 'MagTrain/readings.csv'
        testY = 'MagTest/angsrad.csv'
        testX = 'MagTest/readings.csv'
    
    training_data = WindAngDataset(trainY, trainX, transform=None)
    testing_data = WindAngDataset(testY, testX, transform=None)
    epochs = epochs0
    
elif dataSetType==3:
    trainY1 = 'CrossTrain/crossmags.csv'
    trainY2 = 'CrossTrain/crossangsrad.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY1 = 'CrossTest/crossmags.csv'
    testY2 = 'CrossTest/crossangsrad.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = CrossWireDataset(trainY1, trainY2, trainX, transform=None)
    testing_data = CrossWireDataset(testY1, testY2, testX, transform=None)
    epochs = 2*epochs0
    
    crosswireVal = True
    
elif dataSetType==4:
    
    trainY = 'CrossTrain/crossmags.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY = 'CrossTest/crossmags.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = WindMagDataset(trainY, trainX, transform=None)
    testing_data = WindMagDataset(testY, testX, transform=None)
    epochs = epochs0
    
elif dataSetType==5:
    trainY = 'CrossTrain/crossangsrad.csv'
    trainX = 'CrossTrain/crossreadings.csv'
    testY = 'CrossTest/crossangsrad.csv'
    testX = 'CrossTest/crossreadings.csv'
    
    training_data = WindAngDataset(trainY, trainX, transform=None)
    testing_data = WindAngDataset(testY, testX, transform=None)
    epochs = epochs0
    
    fullAnglesVal = True
    
else:
    raise ValueError('Not a valid dataSetType index (must be in {1, 2, 3, 4, or 5})')
    
'''
# Make training and testing data
train_dataloader = DataLoader(training_data, batch_size=180, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=72, shuffle=True)
model = NeuralNetwork(crosswire=crosswireVal, fullAngles=fullAnglesVal, geom=geometryVal)
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)
'''